In [ ]:
# commands to generate datasets to be converted here (maybe lower the memroy)
#  bsub -R "rusage[mem=250000]" -W 24:00 
# bsub ./a.out --gene-len 100 --gene-len2 2000 -G 100 -P 10000 -n 3 -r 2 --geometric-p .5 -m .1  -f data1
# bsub -R "rusage[mem=99000]" -W 24:00 ./a.out --gene-len 100 --gene-len2 2000 -G 3000 -P 55000 -n 3 -r 3 --geometric-p .34 -m .1  -f data16
# bsub -R "rusage[mem=200000]" -W 24:00  ./a.out --gene-len 100 --gene-len2 2000 -G 6000 -P 55000 -n 3 -r 3 --geometric-p .34 -m .1  -f data17
# bsub -R "rusage[mem=250000]" -W 24:00  ./a.out --gene-len 100 --gene-len2 2000 -G 12000 -P 55000 -n 3 -r 3 --geometric-p .34 -m .1 -f data18
# bsub -R "rusage[mem=500000]" -W 24:00  ./a.out --gene-len 100 --gene-len2 3000 -G 24000 -P 55000 -n 3 -r 3 --geometric-p .34 -m .1 -f data19

import numpy as np
import glob
from attrdict import AttrDict
import utility

file_name = 'data16'
proj_dir = utility.proj_dir() + '/data/fasta/'
datadir = proj_dir + file_name

# read maf file 
fasta_files = glob.glob(datadir+'/seqs*.fa')
N = len(fasta_files)
genes = [[] for i in range(N)]
maf_file = open(datadir + '/MSA.maf','r')
line = maf_file.readline()
while '===' not in line:
    if line[0]=='s':
        line = maf_file.readline()
        continue
    seq, g, s, l = [int(i) for i in line.split()]
    genes[seq].append([g, s, l])
    line = maf_file.readline()
    
# read fasta files and fill the values 
seqs = []
vals = []

for i in range(N):
    f = open(fasta_files[i])
    f.readline()
    seq = f.readline()[:-1]
    L = len(seq)
    val = np.zeros(L, dtype=np.uint32)
    for gene in genes[i]:
        gc, s, l = gene
        val[s:s+l] = gc
    seqs.append(seq)
    vals.append(val)
    
    
    
# read options file 

opts_files = open(datadir+'/options.txt')
lines = opts_files.readlines()
for line in lines:
    if 'num_seqs' in line:
        num_seqs = [int(i) for i in line.split()[1:]]
    elif 'gene_lens' in line:
        gene_lens = [int(i) for i in line.split()[1:]]
    elif 'seq_options' in line:
        opts = line.replace(',', ' ').replace('}','').replace('{','').split()[2:]
        Dict = dict()
        for i in range(int(len(opts)/2)):
            key, val = opts[2*i:2*i +2]
            if key in ['mutation_rate', 'geometric_p', 'reverse_p']:
                Dict[key] = float(val)
            elif key == 'file_name':
                Dict['save_directory'] = val
            else:
                Dict[key] = int(val)
Dict['num_seqs'] = num_seqs
Dict['gene_lens'] = gene_lens
options = AttrDict(Dict)    

        
# save the values
np.savez(utility.proj_dir() + '/data/'+file_name+'.npz', seqs = seqs, vals = vals, num_seqs = num_seqs, gene_lens = gene_lens, options = Dict )

In [ ]:
import aggregate

quads = aggregate.get_quads(12,'experimental/data1')

genes = set()
for q in quads:
    genes.add(int(q[1][1]))
print(genes)
Gene_lens = [gene_lens[(g-1)% options.num_genes] for g in genes]
import matplotlib.pyplot as plt
print(sorted(Gene_lens))

In [ ]:
len(quads)/(2*3*2*100)